In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install datasets numpy
!pip install SupCL_Seq
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 32.2 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 2.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-no

In [3]:
!nvidia-smi

Wed May 15 12:08:33 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
from datasets import load_metric, DatasetDict, Dataset, ClassLabel, load_from_disk, concatenate_datasets
from transformers import TrainingArguments, Trainer, AutoTokenizer, AutoModel, AutoModelForSequenceClassification
import warnings
import numpy as np
import os
import re

from SupCL_Seq import SupCsTrainer

warnings.filterwarnings('ignore')

##Model Setup

In [9]:
model_path = '/content/drive/MyDrive/FYP/models-final/KanoonBert'

tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=True)
model = AutoModel.from_pretrained(model_path)

## Data Preparation


###Training Dataset

In [10]:
old_train_data = load_from_disk('/content/drive/MyDrive/FYP/Dataset/dataset_final')['train']

In [11]:
train_dataset = []
directory = '/content/drive/MyDrive/FYP/Dataset/encoded_dataset'
for file in os.listdir(directory):
    file_name = os.path.join(directory, file)
    e_data = load_from_disk(file_name)

    match = int(re.search(r'\d+', file_name).group())
    o_data = old_train_data[match:match+4]
    label_col = o_data['label']

    new_train_data = e_data.add_column('label',label_col)
    train_dataset.append(new_train_data)

In [12]:
final_train_dataset = concatenate_datasets(train_dataset)

In [13]:
print(final_train_dataset)

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'label'],
    num_rows: 3702
})


###Validation Dataset

In [14]:
old_val_data = load_from_disk('/content/drive/MyDrive/FYP/Dataset/dataset_final')['validation']

In [15]:
val_dataset = []
directory = '/content/drive/MyDrive/FYP/Dataset/encoded_dataset_validation'
for file in os.listdir(directory):
    file_name = os.path.join(directory, file)
    e_val_data = load_from_disk(file_name)

    match = int(re.search(r'\d+', file_name).group())
    o_val_data = old_val_data[match:match+4]
    label_col = o_val_data['label']

    new_val_data = e_val_data.add_column('label',label_col)
    val_dataset.append(new_val_data)

In [16]:
final_val_dataset = concatenate_datasets(val_dataset)

In [17]:
print(final_val_dataset)

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'label'],
    num_rows: 924
})


##Task Setup

In [18]:
metric = load_metric('glue', 'mrpc')

# Used in the trainers
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

##Training Setup

In [ ]:
CL_args = TrainingArguments(
        output_dir = '/content/drive/MyDrive/FYP/fine-tuning/models/output',
        # save_total_limit = 1,
        num_train_epochs=5,
        per_device_train_batch_size=2,
        evaluation_strategy = 'no',
        logging_steps = 50,
        learning_rate = 5e-05,
        warmup_steps=50,
        report_to ='tensorboard',
        weight_decay=0.01,
        logging_dir='/content/drive/MyDrive/FYP/fine-tuning/models/logs',
    )

In [ ]:
SupCL_trainer = SupCsTrainer.SupCsTrainer(
    w_drop_out=[0.0, 0.1, 0.2],
    temperature=0.05,
    def_drop_out=0.1,
    pooling_strategy='pooler',
    model=model,
    args=CL_args,
    train_dataset=final_train_dataset,
    eval_dataset=final_val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Employing pooler ([CLS]) output.


In [ ]:
SupCL_trainer.train()

Step,Training Loss
50,2.084100
100,1.236200
150,1.199400
200,1.015300
250,1.180700
300,1.033700
350,1.199700
400,1.237900
450,1.206700
500,1.100300


Step,Training Loss
50,2.084100
100,1.236200
150,1.199400
200,1.015300
250,1.180700
300,1.033700
350,1.199700
400,1.237900
450,1.206700
500,1.100300


TrainOutput(global_step=9255, training_loss=1.102652887522627, metrics={'train_runtime': 7827.8731, 'train_samples_per_second': 2.365, 'train_steps_per_second': 1.182, 'total_flos': 4870098179850240.0, 'train_loss': 1.102652887522627, 'epoch': 5.0})

In [ ]:
SupCL_trainer.save_model('/content/drive/MyDrive/FYP/fine-tuning/models/fine-tuned-model')

##Single Layer Training

In [19]:
finetuned_model_path = '/content/drive/MyDrive/FYP/fine-tuning/models/fine-tuned-model'#"./results/checkpoint-500/"
num_labels =  2

#------ Add classification layer ---------#
#model = RobertaForSequenceClassification.from_pretrained(model_name,num_labels=num_labels)
fine_tuned_model = AutoModelForSequenceClassification.from_pretrained(finetuned_model_path,num_labels=num_labels)
# ---- Freeze the base model -------#
for param in fine_tuned_model.base_model.parameters():
    param.requires_grad = False

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/FYP/fine-tuning/models/fine-tuned-model and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
args = TrainingArguments(
        output_dir = '/content/drive/MyDrive/FYP/fine-tuning/models/results',
        save_total_limit = 1,
        num_train_epochs=5,
        per_device_train_batch_size=28,
        per_device_eval_batch_size=64,
        evaluation_strategy = 'epoch',
        logging_steps = 200,
        learning_rate = 1e-03,
        eval_steps = 200,
        warmup_steps=50,
        report_to ='tensorboard',
        weight_decay=0.01,
        logging_dir='/content/drive/MyDrive/FYP/fine-tuning/models/logs_2',
    )

In [21]:
trainer = Trainer(
            fine_tuned_model,
            args,
            train_dataset=final_train_dataset,
            eval_dataset=final_val_dataset,
            tokenizer=tokenizer,
            compute_metrics=compute_metrics
        )

In [22]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.002373,1.000000,1.000000
2,0.057600,0.000696,1.000000,1.000000
3,0.057600,0.006326,1.000000,1.000000
4,0.015200,0.000732,1.000000,1.000000
5,0.015600,0.000884,1.000000,1.000000


TrainOutput(global_step=665, training_loss=0.02801147717282288, metrics={'train_runtime': 879.9271, 'train_samples_per_second': 21.036, 'train_steps_per_second': 0.756, 'total_flos': 4870185634713600.0, 'train_loss': 0.02801147717282288, 'epoch': 5.0})

In [23]:
trainer.save_model('/content/drive/MyDrive/FYP/fine-tuning/models/fine-tuned-model-single-layer')